In [1]:
# %matplotlib
%load_ext autoreload
%autoreload 2
from functions4 import *
from declarations2 import *
from classes4 import *
import numpy as np
import SkymapPlot.SkyMap
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from astropy.io import fits
#from matplotlib import rcParams
#rcParams.update({'figure.autolayout': True})
#import plotly.plotly as py
SAVE_DIRECTORY = "/net/scratch_icecube4/user/kalaczynski/Plots/"
l_max=1000



running on machine: lx3b89.physik.rwth-aachen.de


In [2]:
# for broader display:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Running the main code

In [38]:
# DETECTOR="0001"  # "IC86_12_13_14" ,"IC86_11", "IC79" "IC59"   | Binary System
#                                                                    mu  #src
%run createSmearedSkymap_multipleyears.py "multi_mu_1" 0 1 "VarSig" [30] [0] detector="1111" gamma=2.13 catalog="NVSS" useE=False useAlm=False Renorm=False UseGalPlane=True OnlyGalPlane=False 
#Theta=[130.489164] Phi=[299.259131]
#extended_source=2.
#useCatalogPos=10
#useweakestsource=10
#useDiffBG=True
#OnlyGalPlane=True

running on machine: lx3b89.physik.rwth-aachen.de
DETECTOR Configuration:
['IC86_12_13_14', 'IC86_11', 'IC79', 'IC59']
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_12_13_14 the total uptime is 91563185.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2012_13_14_11029_11069_11070_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC86_11 the total uptime is 29556432.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC86_2011_10602_11077_11191_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC79 the total uptime is 26784000.0 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC79_6308_6850_corrected_ow.npy
/net/scratch_icecube4/user/glauch/DATA/
For Detector Configuration: IC59 the total uptime is 30079123.2 s
Read MC: /net/scratch_icecube4/user/glauch/DATA/IC59_6471_6515_corrected_ow.npy
nside as usual: 512
Number of Events per Chosen Sample:
[217033, 71103, 36851, 21442]
Reading 

# FITS FILES

In [30]:
#FITS files:
filename="COM_PCCS_030_R2.04"
path="/net/scratch_icecube4/user/kalaczynski/Catalogs/COM_PCCS-Catalogs_vPR2/"+str(filename)+".fits"
hdulist = fits.open(path)
#hdulist
#fitsMap=H.read_map(path)
#test=H.mrdfits(path)
#print test[1]

In [4]:
print hdulist[1].data

In [18]:
H.mollview(test[1], coord=['G','E'], title='Histogram equalized Ecliptic', unit='mK', norm='hist', min=-1,max=1, xsize=2000)
H.graticule()
plt.show()

In [6]:
#print len(catalog_map)
#print len(fitsMap)
#print len(mergeGPU.map_delta)
#print len(fitsMap)/len(catalog_map)
#fitsAlm=[]
#print path
#fitsAlm=H.read_alm(path)
#print len(fitsAlm)

In [7]:
H.gnomview(fitsMap, rot=[0,0.3], title='GnomView', unit='mK', format='%.2g')
plt.show()

In [17]:
H.mollview(mergeGPU.map_delta, rot=[0,0.3], title='GnomView', unit='mK', format='%.2g')
plt.show()

In [42]:
print sum(mergeGPU.map_delta)
print sum(catalog_map*2)

346429.0
176.343412972


# Plotting skymap:

In [40]:
nFigure=5
Smoothing=True
Galactic=False
Catalog=True
NewMap=False
logz=True
plot=newfig(5., 3.)
Norm=None
if logz: Norm='log'    

if NewMap==True:
    usedMap=newMap
else:
    if Catalog==False:
        usedMap=mergeGPU.map_delta
    else:
        usedMap=catalog_map

#usedMap=fitsMap       
#usedMap=modif    
#usedMap=map_delta2

if Smoothing:
    smoothed_map = H.smoothing(usedMap, sigma=np.radians(0.5))
else:
    smoothed_map=usedMap
if min(smoothed_map)<=0:
    raiser=abs(min(smoothed_map))*2
else:
    raiser = 0.
    
for i in range(len(smoothed_map)):
    smoothed_map[i] = float(smoothed_map[i])
    if logz:
        smoothed_map[i]=smoothed_map[i]+raiser
       

colormap=plt.get_cmap("Blues")
#colormap.set_under("white")
colormap.set_bad("white")
#col2="0."
#col="0."
res=800

smoothed_map= np.ma.masked_array(smoothed_map)
#smoothed_map.mask = np.log10(smoothed_map) < 18.1  #####BE careful..Value hardcoded
smoothed_map.mask = smoothed_map == 0

if Catalog:
    plotTitle=r"$\log_{10}(\frac{\phi}{\phi_{\mathrm{Total}}} \cdot A_{\mathrm{eff}})$"
    #plotTitle=r"$\log_{10}(\frac{\mathrm{Flux}}{\mathrm{Total Flux}} \cdot A_{\mathrm{eff}})$"
else:
    plotTitle=r"$\log_{10}(N_{\mathsf{Events}})$"

if Galactic==False:
    skymap=SkymapPlot.SkyMap.SkyMap(fig=plot)
    #smoothed_map=usedMap
    skymap.plot_map(np.log10(smoothed_map),
                    cmap=colormap, npix=res, 
                    colorbar=dict(title=plotTitle))
    if (Catalog!=True) and len(mergeGPU.phi_sou)>0:
        skymap.plot_circle(mergeGPU.phi_sou,mergeGPU.theta_sou)
    #skymap.fig.savefig(SAVE_DIRECTORY+"BGD_smoothed_equatorial.png", dpi=300, bbox_inches="tight")
else:
    skymap=SkymapPlot.SkyMap.SkyMapGalactic()
    skymap.add_plane()
    #smoothed_map=usedMap
    skymap.plot_map(np.log10(smoothed_map),
                    cmap=colormap, npix=res, 
                    colorbar=dict(title=plotTitle))
    if (Catalog!=True) and len(mergeGPU.phi_sou)>0:
        skymap.plot_circle(mergeGPU.phi_sou,mergeGPU.theta_sou)
    #skymap.fig.savefig(SAVE_DIRECTORY+"BGD_smoothed_galactic.png", dpi=300, bbox_inches="tight")
plt.show()

Sigma is 30.000000 arcmin (0.008727 rad) 
-> fwhm is 70.644601 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin


# Getting alms from the files:

In [17]:
BGD_DIR="/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/No_plane/VarSig_lmax="+str(l_max)+"/0src/Detector=1111"  

In [4]:
# BGD

size=int((1.+(l_max))*((2.+(l_max))/2.)) #size of the alm array
no_files=1000 #max is 1000 so far
temp1=[]
temp2=[]
Alm_BGD=[]
Alm_BGD_avg=[0]*size
  
#BGD files loop:
for i in range(no_files):
    temp1=np.loadtxt(BGD_DIR+"/almAllAbs_"+str(i)+".txt")
    temp2=np.loadtxt(BGD_DIR+"/almAllPhase_"+str(i)+".txt")
    Alm_BGD.append(temp1*np.cos(temp2)+temp1*np.sin(temp2)*1j)
    Alm_BGD_avg=Alm_BGD_avg+((temp1*np.cos(temp2)+temp1*np.sin(temp2)*1j))/no_files

# save BGD_avg as files:
np.savetxt(BGD_DIR+"/Alm_BGD_avg_re.txt", np.real(Alm_BGD_avg))
np.savetxt(BGD_DIR+"/Alm_BGD_avg_im.txt", np.imag(Alm_BGD_avg))

In [46]:
#single readout:
temp1=np.loadtxt(BGD_DIR+"/almAllAbs_0.txt")
temp2=np.loadtxt(BGD_DIR+"/almAllPhase_0.txt")
Alm_BGD=temp1*np.cos(temp2)+temp1*np.sin(temp2)*1j

In [7]:
# save catalog alms:
Alm_cat=H.anafast(catalog_map, lmax=l_max, alm="True")[1]
np.savetxt(BGD_DIR+"/CatAlm_re.txt", np.real(Alm_cat))
np.savetxt(BGD_DIR+"/CatAlm_im.txt", np.imag(Alm_cat))

In [5]:
# read catalog files:
re_part=np.loadtxt(BGD_DIR+"/CatAlm_re.txt")
im_part=np.loadtxt(BGD_DIR+"/CatAlm_im.txt")
Alm_cat=re_part+im_part*1j

IOError: [Errno 2] No such file or directory: '/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/No_plane/VarSig_lmax=1000/0src/Detector=1111/CatAlm_re.txt'

In [4]:
# read BGD_avg files:
re_part=np.loadtxt(BGD_DIR+"/Alm_BGD_avg_re.txt")
im_part=np.loadtxt(BGD_DIR+"/Alm_BGD_avg_im.txt")
Alm_BGD_avg=re_part+im_part*1j

In [8]:
#read signal files:
no_srcs=5
used_mu=30
SIG_DIR="/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/No_plane/VarSig_lmax="+str(l_max)+"/"+str(no_srcs)+"src_mu"+str(used_mu)+"/Detector=1111"
Alm_VarSig=[]

abs_sig=np.loadtxt(SIG_DIR+"/almAllAbs_0.txt")
phase_sig=np.loadtxt(SIG_DIR+"/almAllPhase_0.txt")

Alm_VarSig=abs_sig*np.cos(phase_sig)+abs_sig*np.sin(phase_sig)*1j


# Alm plotting

In [6]:
plt.xlabel("Im($a^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)
plt.ylabel("Re($a^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)
#plt.scatter(np.imag(Alm_VarSig*10**3), np.real(Alm_VarSig*10**3), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='g')
plt.scatter(np.imag(Alm_BGD[0]*10**6), np.real(Alm_BGD[0]*10**6), label="BGD without $m=0$, $l_{\mathsf{max}}=$"+str(l_max), color='r')
plt.scatter(np.imag(Alm_BGD_no_m0[0]*10**6), np.real(Alm_BGD_no_m0[0]*10**6), label="BGD without $m=0$, $l_{\mathsf{max}}=$"+str(l_max), color='g')

plt.legend()
plt.gcf().set_size_inches(15, 11)
plt.savefig(SAVE_DIRECTORY+"alm_vs_blm_test.png", dpi=300)
plt.show()


In [42]:
# Plotting alm vs blm:
plt.xlabel("Re($a^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)
plt.ylabel("Re($b^{l}_{m}$)$\cdot 10^{6}$", fontsize=34)

plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_VarSig6*10**6), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='r')
plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_VarSig4*10**6), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='b')
plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_VarSig4_noBG*10**6), label="VarSig, $l_{\mathsf{max}}=$"+str(l_max), color='g')
#plt.scatter(np.real(Alm_cat*10**6), np.real(Alm_cat*10**6), label="Catalogue, $l_{\mathsf{max}}=$"+str(l_max), color='magenta')


plt.legend()
plt.gcf().set_size_inches(15, 11)
#plt.savefig(SAVE_DIRECTORY+"almVsBlmTestImag.png", dpi=300)
plt.show()


In [169]:
#histogram with Gaussian fit:
num_bins = 30
plt.xlabel("Re($a^{l}_{m}$)$\cdot 10^{5}$", fontsize=34)
plt.ylabel("Amplitude [a.u.]", fontsize=34)


x=np.abs(Alm_VarSig3)*100000
mu = np.mean(x) # mean
sigma = np.std(x) # standard deviation 
n, bins, patches = plt.hist(x,num_bins, normed = True, label="BGD, $l_{\mathsf{max}}=$"+str(l_max))
y = mlab.normpdf(bins, mu, sigma)
plt.plot(bins, y, 'r--', label="Gauss fit")
plt.legend()
plt.gcf().set_size_inches(15, 11)
plt.savefig(SAVE_DIRECTORY+"histo_BGD_Re.png", dpi=300)
plt.show()

# Cross-correlation:

In [51]:
no_srcs=5
used_mu=30
SIG_DIR="/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/Only_plane/VarSig_lmax="+str(l_max)+"/"+str(no_srcs)+"src_mu"+str(used_mu)+"/Detector=1111"
BGD_DIR="/net/scratch_icecube4/user/kalaczynski/Analysis_stuff/Normed/Only_plane/VarSig_lmax="+str(l_max)+"/0src/Detector=1111"

In [16]:
# save catalog alms:
selfCat=H.alm2cl(Alm_cat, lmax=1000)
np.savetxt(BGD_DIR+"/CatCl.txt", selfCat)

In [ ]:
# save avg Cl's:
np.savetxt(BGD_DIR+"/ClAutoAll_avg.txt", selfBGD_avg)
np.savetxt(BGD_DIR+"/ClAll_avg.txt", CatXBGD_avg)
np.savetxt(SIG_DIR+"/ClAll_avg.txt", CatXVarSig_avg)
np.savetxt(SIG_DIR+"/ClAutoAll_avg.txt", selfVarSig_avg)

In [5]:
# Calculate Cl’s from alms
selfCat=H.alm2cl(Alm_cat, lmax=1000)
selfBGD=H.alm2cl(Alm_BGD_avg, lmax=1000)
CatXBGD=H.alm2cl(Alm_cat, Alm_BGD_avg, lmax=1000)
#CatXVarSig=H.alm2cl(Alm_cat, Alm_VarSig, lmax=1000)

running on machine: lx3b89.physik.rwth-aachen.de


NameError: name 'Alm_cat' is not defined

In [52]:
#read Cl’s from files:
no_files=1000 #max is 1000 so far
selfBGD=[]
CatXBGD=[]
CatXVarSig=[]
selfVarSig=[]

selfCat=np.loadtxt(BGD_DIR+"/CatCl.txt")
for i in range(no_files):
    selfBGD.append(np.loadtxt(BGD_DIR+"/ClAutoAll_"+str(i)+".txt"))
    CatXBGD.append(np.loadtxt(BGD_DIR+"/ClAll_"+str(i)+".txt"))
    CatXVarSig.append(np.loadtxt(SIG_DIR+"/ClAll_"+str(i)+".txt"))
    selfVarSig.append(np.loadtxt(SIG_DIR+"/ClAutoAll_"+str(i)+".txt"))
selfBGD_avg=np.average(selfBGD, axis=0)
CatXBGD_avg=np.average(CatXBGD, axis=0)
CatXVarSig_avg=np.average(CatXVarSig, axis=0)
selfVarSig_avg=np.average(selfVarSig, axis=0)

In [16]:
# Cl's from maps:
selfCat=H.anafast(catalog_map, lmax=1000)

In [7]:
print sum(selfCat)
print sum(selfBGD_avg)
print sum(CatXBGD_avg)
print sum(CatXVarSig_avg)
print sum(selfVarSig_avg)

0.431981325639
148.751833789
0.44612842107
0.442874737699
145.21775824


# Plotting $C_l$

In [5]:
llim=1000+1

In [71]:
# Parameters for error bands:
x=range(llim)
mean0=selfCat[:llim]
sigma0=np.std(selfCat[:llim])
mean1=selfBGD_avg[:llim]
sigma1up=np.amax(selfBGD[:llim], axis=0)
sigma1down=np.amin(selfBGD[:llim], axis=0)
#sigma1up=np.std(selfBGD[:llim]) # alternative with 1 standard deviation as error
#sigma1down=np.std(selfBGD[:llim])
mean2=selfVarSig_avg[:llim]
sigma2up=np.amax(selfVarSig[:llim], axis=0)
sigma2down=np.amin(selfVarSig[:llim], axis=0)
#sigma2up=np.std(selfVarSig[:llim])
#sigma2down=np.std(selfVarSig[:llim])
mean3=CatXBGD_avg[:llim]
sigma3up=np.amax(CatXBGD[:llim], axis=0)
sigma3down=np.amin(CatXBGD[:llim], axis=0)
#sigma3up=np.std(CatXBGD[:llim])
#sigma3down=np.std(CatXBGD[:llim])
mean4=CatXVarSig_avg[:llim]
sigma4up=np.amax(CatXVarSig[:llim], axis=0)
sigma4down=np.amin(CatXVarSig[:llim], axis=0)
#sigma4up=np.std(CatXVarSig[:llim])
#sigma4down=np.std(CatXVarSig[:llim])

In [49]:
# Adding curves:
x=np.array(range(llim))

plt.scatter(x,x*(x+1)*selfCat[:llim], label="Cat x Cat", color = 'black', s=2)
#plt.fill_between(x, mean0+sigma0, mean0-sigma0, color='black', alpha=0.5)

plt.scatter(x,x*(x+1)*selfBGD_avg[:llim], label="BGD x BGD", color = 'cyan', s=2)
#plt.fill_between(x, abs(mean1+sigma1up), abs(mean1-sigma1down), color='cyan', alpha=0.5)
#plt.fill_between(x, abs(sigma1up), abs(sigma1down), color='cyan', alpha=0.5, where = sigma1down > 0)

plt.scatter(x,x*(x+1)*selfVarSig_avg[:llim], label="VarSig x VarSig", color = 'magenta', s=2)
#plt.fill_between(x, abs(mean2+sigma2up), abs(mean2-sigma2down), color='magenta', alpha=0.5)
#plt.fill_between(x, abs(sigma2up), abs(sigma2down), color='magenta', alpha=0.5, where = sigma2down > 0)

plt.scatter(x,x*(x+1)*CatXBGD_avg[:llim], label="Cat x BGD", color = 'blue', s=2)
#plt.fill_between(x, mean3+sigma3up, mean3-sigma3down, color='blue', alpha=0.5)
#plt.fill_between(x, sigma3up, sigma3down, color='blue', alpha=0.5, where = sigma3down > 0)

plt.scatter(x,x*(x+1)*CatXVarSig_avg[:llim], label="Cat x VarSig", color = 'red', s=2)
#plt.fill_between(x, mean4+sigma4up, mean4-sigma4down, color='red', alpha=0.5)
#plt.fill_between(x, sigma4up, sigma4down, color='red', alpha=0.5, where = sigma4down > 0)
# excessive error bands can be removed with np.clip or by inserting  where = mean3 - sigma3down > 0
# but it just looks bad

In [18]:
print len(CatXBGD_avg[CatXBGD_avg<0]), len(CatXBGD_avg[CatXBGD_avg>0])
print len(CatXVarSig_avg[CatXVarSig_avg<0]), len(CatXVarSig_avg[CatXVarSig_avg>0])
print selfCat[4]

494 507
449 552
9.43697725542e-50


In [50]:
#plt.fill_between(selfCat, selfCat-selfCat_err, selfCat+selfCat_err) #looks very bad
#plt.axhline(y=0, xmin=0, xmax=1000, linewidth=2, color = 'black')
plt.xlabel("$l$", fontsize=34)
plt.ylabel("$C_{l}$", fontsize=34)
plt.xlim([0, llim+1])
plt.ylim([10**(-12), 10**(0)])
#plt.ylim([-0.000005, 0.000005])

plt.yscale('log', basey = 10)
plt.legend(fontsize=30, markerscale=10, loc=1)
plt.gcf().set_size_inches(15, 11)
#plt.savefig(SAVE_DIRECTORY+"Cl_only_plane_zoom_l200.png", dpi=300) #_zoom_l200 _selfBGD _errbands _negative
plt.show()

In [30]:
# single ones
#plt.scatter(range(len(selfCat)),selfCat, label="Cat x Cat", color = 'black', s=2)
#plt.scatter(range(len(CatXBGD_avg)),CatXBGD_avg, label="Cat x BGD avg", color = 'red', s=2)
for i in range(1000):
    if i==3:
        plt.scatter(range(len(selfBGD[i][:llim])),selfBGD[i][:llim], color = 'cyan', s=1, label="BGD x BGD")
    plt.scatter(range(len(selfBGD[i][:llim])),selfBGD[i][:llim], color = 'cyan', s=1)
    
    #plt.scatter(range(len(selfVarSig[i][:llim])),selfVarSig[i][:llim], color = 'red', s=1)
#plt.scatter(range(len(CatXVarSig_avg)),CatXVarSig_avg, label="Cat x VarSig", color = 'green', s=2)

# $C_l$ distribution

In [7]:
# generate wanted C_l spectrta:

no_files=1000 #max is 1000 so far
temp=[]
Cl5=[]
Cl50=[]
Cl150=[]
Cl500=[]
  
#BGD files loop:
for i in range(no_files):
    temp=np.loadtxt(BGD_DIR+"/ClAutoAll"+str(i)+".txt")
    Cl5.append(temp[5])
    Cl50.append(temp[50])
    Cl150.append(temp[150])
    Cl500.append(temp[500])

# save as files:
np.savetxt(BGD_DIR+"/Cl5.txt", Cl5)
np.savetxt(BGD_DIR+"/Cl50.txt", Cl50)
np.savetxt(BGD_DIR+"/Cl150.txt", Cl150)
np.savetxt(BGD_DIR+"/Cl500.txt", Cl500)

In [15]:
# reading the spectra:
# Cl5=np.loadtxt(BGD_DIR+"/Cl5.txt")
Cl50=np.loadtxt(BGD_DIR+"/Cl50.txt")
Cl150=np.loadtxt(BGD_DIR+"/Cl150.txt")
Cl500=np.loadtxt(BGD_DIR+"/Cl500.txt")

In [16]:
#histogram:
num_bins = 100
plt.xlabel("$C_{l}$", fontsize=34)
plt.ylabel("Amplitude [a.u.]", fontsize=34)


#x=Cl5
#mu = np.mean(x) # mean
#sigma = np.std(x) # standard deviation 
#n, bins, patches = plt.hist(x,num_bins, normed = True, label="$C_{l=0}$")
#y = mlab.normpdf(bins, mu, sigma)

x2=Cl50
mu2 = np.mean(x2) # mean
sigma2 = np.std(x2) # standard deviation 
n2, bins2, patches2 = plt.hist(x2,num_bins, normed = True, label="$C_{l=50}$", edgecolor= "none")
#y2 = mlab.normpdf(bins2, mu2, sigma2)

x3=Cl150
mu3 = np.mean(x3) # mean
sigma3 = np.std(x3) # standard deviation 
n3, bins3, patches3 = plt.hist(x3,num_bins, normed = True, label="$C_{l=150}$", edgecolor= "none")

x4=Cl500
mu4 = np.mean(x4) # mean
sigma4 = np.std(x4) # standard deviation 
n4, bins4, patches4 = plt.hist(x4,num_bins, normed = True, label="$C_{l=500}$", edgecolor= "none")

#plt.plot(bins, y, 'r--', label="Gauss fit")
#plt.plot(bins2, y2, 'r--', label="Gauss fit2")
plt.legend()
plt.gcf().set_size_inches(15, 11)
plt.savefig(SAVE_DIRECTORY+"histo_Cl_onlyPlane.png", dpi=300)
plt.show()

# Similarity comparison

In [9]:
# max similarity:1, min: 0        # for now it is just totally prototype phase
def similarity(arg1,arg2):
    sim=1.-sum(abs(arg1-arg2))/sum(0.5*(arg1+arg2))
    return sim
S1=similarity(CatXVarSig,selfCat)
S2=similarity(CatXBGD,selfCat)
S3=similarity(selfBGD,selfCat)
#print S1
#print S2
#print S3

In [10]:
#Nash–Sutcliffe:     (equvalent to Coefficient of determination (R squared))
def nash_sutcliffe(arg1,cat):
    arg1_avg=np.average(arg1)
    sim=1.-sum((arg1-cat)**2)/sum((arg1-arg1_avg)**2)
    return sim
S1=nash_sutcliffe(CatXVarSig,selfCat)
S2=nash_sutcliffe(CatXBGD,selfCat)
S3=nash_sutcliffe(selfBGD,selfCat)
#print S1
#print S2
#print S3

In [11]:
# np. correlate: (cross-corellation)
S1=np.correlate(CatXVarSig,selfCat)
S2=np.correlate(CatXBGD,selfCat)
S3=np.correlate(selfBGD,selfCat)
S4=np.correlate(selfCat,selfCat)
#print S1
#print S2
#print S3
#print S4

In [12]:
#the covariation method:
def covariation(arg1,cat):
    sim=1./len(arg1)*sum(abs(1.-arg1/cat))
    return sim
S1=covariation(CatXVarSig,selfCat)
S2=covariation(CatXBGD,selfCat)
S3=covariation(selfBGD,selfCat)
#print S1
#print S2
#print S3

# Resolution tests etc.

In [8]:
# max reasonable l:
test=H.nside2resol(512)
print 180/np.degrees(test)
print 3*512-1
print np.degrees(test)
test2=H.nside2pixarea(512, degrees=True)
print test2
nside=H.nside2npix(512)
print nside, 12*512*512

1571.82982341
1535
0.114516213727
0.0131139632064
3145728 3145728


In [25]:
#atmGPU[id].plotLastSkymap(title=SIGNAL+", "+SMEARING_METHOD, nFigure=1, logz=False)
nFigure=7
title=SIGNAL+", "+SMEARING_METHOD

grid=True
logz=True

plotControl(nFigure)
Norm=None
if logz: Norm='log'

smoothed_map = H.smoothing(catalog_map, sigma=np.radians(0.5))
if min(smoothed_map)<=0:
    raiser=abs(min(smoothed_map))*2
else:
    raiser = 0.
    
for i in range(len(smoothed_map)):
    smoothed_map[i] = float(smoothed_map[i])
    if logz:
        smoothed_map[i]=smoothed_map[i]+raiser #does not affect effCl, only overall rise

colormap=plt.get_cmap("Blues")
colormap.set_under("1")
col2="0."
col="0."
res=800

H.mollview(smoothed_map, cmap=colormap, title="", fig=nFigure, xsize=res, norm=Norm) #H.smoothing(, sigma=np.radians(0.5)
H.graticule(dpar=30,dmer=60)


H.projplot(zen2dec(mergeGPU.theta_sou), mergeGPU.phi_sou, color="orange", alpha=0.3, marker="o", ms=50., mew=2., mec="orange", lw=0.)

if grid:
    
    H.projtext(58.0, -8.0, "$60^{\circ}$",lonlat=True, color=col2)
    H.projtext(118.0, -8.0, "$120^{\circ}$",lonlat=True, color=col2)
    H.projtext(178.0, -8.0,"$180^{\circ}$",lonlat=True, color=col2)
    
    H.projtext(-2.0, -8.0, "$0^{\circ}$",lonlat=True, color=col2)
    H.projtext(-62.0, -8.0, "$300^{\circ}$",lonlat=True, color=col2)
    H.projtext(-122.0, -8.0, "$240^{\circ}$",lonlat=True, color=col2)

    H.projtext(-2.0, 22.0, "$30^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, 52.0, "$60^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, 76.0, "$90^{\circ}$",lonlat=True, color=col)

    H.projtext(-2.0, -28.0, "$-30^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, -58.0, "$-60^{\circ}$",lonlat=True, color=col)
    H.projtext(-2.0, -82.0, "$-90^{\circ}$",lonlat=True, color=col)
plt.show()

Sigma is 30.000000 arcmin (0.008727 rad) 
-> fwhm is 70.644601 arcmin
Sigma is 0.000000 arcmin (0.000000 rad) 
-> fwhm is 0.000000 arcmin
0.0 180.0 -180.0 180.0
The interval between parallels is 30 deg -0.00'.
The interval between meridians is 60 deg -0.00'.


# Checking energy & zenith

In [24]:
plt.figure(3)
#n, bins = np.histogram(np.cos(dec2zen(atmGPU[id].sample_atm["dec"])), bins=80, density=True) #theta values of events in map
#n = normHist(n, 0.95)
#plt.plot(setNewEdges(bins), n, drawstyle="steps-mid", label="cos(sampleTheta) atm")

#.sample_astro["dec"]
n, bins = np.histogram(np.cos(atmGPU[id].ranTheta[:]), bins=80, density=True) #theta values of events in map
n = normHist(n, 0.95)
plt.plot(setNewEdges(bins), n, drawstyle="steps-mid", label="ran theta all", color="green")
plt.legend(loc=2)
plt.show()

In [23]:
plt.figure(4)

n, bins = np.histogram(atmGPU[id].ranPhi[:], bins=80, density=True) #theta values of events in map
n = normHist(n, 0.95)
plt.plot(setNewEdges(bins), n, drawstyle="steps-mid", label="ran phi all", color="red")
plt.legend(loc=2)
plt.show()

In [22]:
# acceptance or sth similar
nmc, binsmc = np.histogram(np.cos(np.array(dec2zen(atmGPU[id].mc_sample_full["dec"]))), bins=100, weights=atmGPU[id].mc_sample_full["conv"], density=True) #theta values of mc atm
nmc = normHist(nmc, 0.95)
plt.plot(setNewEdges(binsmc), nmc, drawstyle="steps-mid", label="cos(fullTheta) atm")

nmc, binsmc = np.histogram(np.cos(np.array(dec2zen(atmGPU[id].mc_sample_full["dec"]))), bins=100, weights=atmGPU[id].mc_sample_full["astro"], density=True) #theta values of mc atm
nmc = normHist(nmc, 0.95)
plt.plot(setNewEdges(binsmc), nmc, drawstyle="steps-mid", label="cos(fullTheta) astro")

plt.legend(loc="lower left", fontsize = 'x-small')
plt.show()

In [20]:
def plotEnergyDistr(nFigure=-1, nLabel=['Diff. Background', 'Atm. Background'], normed=False):
    plotControl(nFigure)
    nColor=["lightblue", "yellow"]

    cut=len(atmGPU[id].diffElist)
    purebg=atmGPU[id].energyList[cut:]

    plt.hist([atmGPU[id].diffElist, purebg], bins= 40, range=(hmin,hmax), stacked=True, color=nColor, label=nLabel, normed=normed)
    plt.semilogy(nonposy="clip")
    plt.legend()
    plt.title("Event Energy Spectrum")
    plt.xlabel(r"log(E)[GeV]")
    plt.ylabel("#")
    
hmin=min(atmGPU[id].energyList)
hmax=max(atmGPU[id].energyList)
print "min energy: "+str(hmin)
print "max energy: "+str(hmax)    
plotEnergyDistr(2, normed=True)
plt.plot(atmGPU[id].atmE_Edges, atmGPU[id].atmEnergySpline(atmGPU[id].atmE_Edges)+atmGPU[id].sigEnergySpline(atmGPU[id].sigE_Edges)/100.)
plt.plot(atmGPU[id].sigE_Edges, atmGPU[id].sigEnergySpline(atmGPU[id].sigE_Edges)/100.)
plt.ylim([1e-5, 1])
plt.xlim([hmin, hmax])
plt.show()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/lib-tk/Tkinter.py", line 1536, in __call__
    return self.func(*args)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/backends/backend_tkagg.py", line 278, in resize
    self.show()
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/backends/backend_tkagg.py", line 349, in draw
    FigureCanvasAgg.draw(self)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/backends/backend_agg.py", line 469, in draw
    self.figure.draw(self.renderer)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/artist.py", line 59, in draw_wrapper
    draw(artist, renderer, *args, **kwargs)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/pyth

min energy: 2.20019
max energy: 6.09067


  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/text.py", line 320, in _get_layout
    ismath=ismath)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/backends/backend_agg.py", line 223, in get_text_width_height_descent
    renderer=self)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/texmanager.py", line 670, in get_text_width_height_descent
    dvifile = self.make_dvi(tex, fontsize)
  File "/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/texmanager.py", line 417, in make_dvi
    report))
RuntimeError: LaTeX was not able to process the following string:
'#'
Here is the full report generated by LaTeX: 

This is pdfTeXk, Version 3.141592-1.40.3 (Web2C 7.5.6)
 %&-line parsing enabled.
entering extended mode
(./5f5fafb36cb15b405f33a20b6adb0520.tex
LaTeX2e <2005/12/01>
Babel <v3.8h

# Skymap

In [18]:
id=2
atmGPU[id].plotLastSkymap(title=SIGNAL+", "+SMEARING_METHOD, nFigure=5, logz=True)
plt.show()

0.0 180.0 -180.0 180.0
The interval between parallels is 30 deg -0.00'.
The interval between meridians is 60 deg -0.00'.


/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/figure.py:1653: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


# Rotation check

In [19]:
from matplotlib.colors import LogNorm
# test plots
plt.figure(13)
f = plt.scatter(np.degrees(atmGPU[id].ps_sample["ra"]), np.degrees(atmGPU[id].ps_sample["dec"]))
#plt.xlim(0, 360)

#plt.ylim(-10,90)
plt.xlabel("RA/deg")
plt.ylabel("Dec/deg")

plt.figure(14)
color = ["blue", "lightgreen", "orange", "red", "magenta", "darkblue", "green", "brown", "darkred", "purple"]
i=0
#colors = ["r","g","b", "c", "orange", "k"]
for rotDec in list(set(atmGPU[id].ps_sample["trueRotDec"])):
    #print i
    plt.scatter(np.degrees(atmGPU[id].ps_sample["trueRa"][np.where(atmGPU[id].ps_sample["trueRotDec"] == rotDec)]), 
                np.degrees(atmGPU[id].ps_sample["trueDec"][np.where(atmGPU[id].ps_sample["trueRotDec"] == rotDec)]),
                color=color[i%10])
    i+=1
plt.xlim(0, 360)
plt.show()